<a href="https://colab.research.google.com/github/Droslj/scATAC-seq-complete-/blob/Google-colab/scATAC_seq_(1)_DA_PyDESeq2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

scATAC seq, based on scATAC seq processing Galaxy tutorials (scATAC preprocessing (2), Standard scATAC seq processing pipeline (1) )
AD Objects created in Galaxy using customized Galaxy WF with Snapatac2 and imported
(1) https://usegalaxy.eu/training-material/topics/single-cell/tutorials/scatac-preprocessing-tenx/tutorial.html#mapping-reads-to-a-reference-genome, (2) https://usegalaxy.eu/training-material/topics/single-cell/tutorials/scatac-standard-processing-snapatac2/tutorial.html
Data taken from the following NCBI study:
Metabolic adaptation pilots the differentiation of human hematopoietic cells (https://www.ncbi.nlm.nih.gov/bioproject/PRJNA1015713)
Import preprocessed Anndata object for four biological replicates, SRR26046013 (cells treated with AOA inhibitor), SRR26046015 (cells treated with DON inhibitor), SRR26046017 (cells treated with DG inhibitor), and SRR26046019 (untreated cells).
Following steps were performed in the preprocessing:
(1) Import matrices
(2) Compute fragment size distribution
(3) Compute TSS enrichment
(4) Filter cell counts based on TSSe
(5) Create cell by bin matrix based on 500 bp wide bins accross the whole genome
(6) Perform feature selection
(7) Perform Doublet removal
(8) Perform Dim reduction (spectral)
(9) Perform Clustering (neighborhood, UMAP, leiden)
(10) Create a cell by gene matrix
(11) Concatenate matrices using Inner join
(12) Remove batch effects

In [1]:
!pip install -q condacolab

In [2]:
import condacolab

In [3]:
condacolab.install()

✨🍰✨ Everything looks OK!


In [4]:
!conda --version

conda 23.11.0


In [5]:
!which conda

/usr/local/bin/conda


In [6]:
!conda config --add channels conda-forge

In [7]:
!conda config --add channels bioconda

In [8]:
!pip install snapatac2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.2/687.2 kB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 60.1 MB/s eta

In [9]:
!pip show snapatac2

Name: snapatac2
Version: 2.8.0
Summary: SnapATAC2: Single-cell epigenomics analysis pipeline
Home-page: https://github.com/
Author: Kai Zhang <kai@kzhang.org>
Author-email: Kai Zhang <zhangkai33@westlake.edu.cn>
License: MIT
Location: /usr/local/lib/python3.10/site-packages
Requires: anndata, igraph, kaleido, macs3, multiprocess, natsort, numpy, pandas, plotly, polars, pooch, pyarrow, pyfaidx, rustworkx, scikit-learn, scipy, tqdm, typeguard
Required-by: 


In [10]:
import snapatac2 as snap

In [11]:
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 62.6 MB/s eta 0:00:00


In [12]:
import umap.umap_ as umap


In [13]:
from umap import UMAP

In [14]:
!pip install scanpy -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━

In [15]:
import scanpy as sc

In [16]:
pip show scanpy

Name: scanpy
Version: 1.10.4
Summary: Single-Cell Analysis in Python.
Home-page: 
Author: Alex Wolf, Philipp Angerer, Fidel Ramirez, Isaac Virshup, Sergei Rybakov, Gokcen Eraslan, Tom White, Malte Luecken, Davide Cittaro, Tobias Callies, Marius Lange, Andrés R. Muñoz-Rojas
Author-email: 
License: 
Location: /usr/local/lib/python3.10/site-packages
Requires: anndata, h5py, joblib, legacy-api-wrap, matplotlib, natsort, networkx, numba, numpy, packaging, pandas, patsy, pynndescent, scikit-learn, scipy, seaborn, session-info, statsmodels, tqdm, umap-learn
Required-by: 


In [33]:
!pip install pydeseq2 -q # Install PyDESeq2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.5 MB/s eta 0:00:00


In [47]:
import pydeseq2

In [54]:
!pip show pydeseq2

Name: pydeseq2
Version: 0.4.12
Summary: A python implementation of DESeq2.
Home-page: 
Author: Boris Muzellec, Maria Telenczuk, Vincent Cabelli and Mathieu Andreux
Author-email: boris.muzellec@owkin.com
License: MIT
Location: /usr/local/lib/python3.10/site-packages
Requires: anndata, matplotlib, numpy, pandas, scikit-learn, scipy
Required-by: 


In [17]:
import numpy as np

In [18]:
import anndata as ad

In [22]:
import matplotlib.pyplot as plt

In [23]:
import seaborn as sns

In [24]:
import plotly.subplots as sp
import plotly.graph_objects as go

In [25]:
from scipy import stats

In [26]:
import pandas as pd

# Import reads from google drive, three samples treated with energy metabolism inhibitors and one untreated

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
SRR26046013_DM_AOA_INH = sc.read_h5ad('/content/drive/MyDrive/Colab Notebooks/SRR26046013_Annotated_data_matrix.h5ad')

In [30]:
SRR26046013_DM_AOA_INH

AnnData object with n_obs × n_vars = 13546 × 0
    obs: 'n_fragment', 'frac_dup', 'frac_mito'
    uns: 'reference_sequences'
    obsm: 'fragment_paired'

In [31]:
SRR26046019_DM_UT = sc.read_h5ad('/content/drive/MyDrive/Colab Notebooks/SRR26046019_Annotated_data_matrix.h5ad')

In [32]:
SRR26046019_DM_UT

AnnData object with n_obs × n_vars = 10448 × 0
    obs: 'n_fragment', 'frac_dup', 'frac_mito'
    uns: 'reference_sequences'
    obsm: 'fragment_paired'

# Perform DeSEQ2 test

In [39]:
gene_anno = '/content/drive/MyDrive/Colab Notebooks/gencode.v41.basic.annotation.gff3.gz'

# Create cell-by-gene matrix (replace with cell-by-bin if appropriate)
gene_matrix_1 = snap.pp.make_gene_matrix(SRR26046013_DM_AOA_INH, gene_anno=gene_anno)
gene_matrix_2 = snap.pp.make_gene_matrix(SRR26046019_DM_UT, gene_anno=gene_anno)

In [40]:
gene_matrix_1

AnnData object with n_obs × n_vars = 13546 × 60606
    obs: 'n_fragment', 'frac_dup', 'frac_mito'

In [41]:
gene_matrix_2

AnnData object with n_obs × n_vars = 10448 × 60606
    obs: 'n_fragment', 'frac_dup', 'frac_mito'

In [42]:
# Extract count matrices and convert to pandas DataFrames
count_matrix_1 = gene_matrix_1.X.toarray()
count_matrix_2 = gene_matrix_2.X.toarray()

In [70]:
gene_matrix_2

AnnData object with n_obs × n_vars = 10448 × 60606
    obs: 'n_fragment', 'frac_dup', 'frac_mito'

In [71]:
# Extract count matrices and convert to pandas DataFrames
count_matrix_1 = gene_matrix_1.X.toarray()
count_matrix_2 = gene_matrix_2.X.toarray()

In [72]:
gene_matrix_2

AnnData object with n_obs × n_vars = 10448 × 60606
    obs: 'n_fragment', 'frac_dup', 'frac_mito'

In [73]:
# Extract count matrices and convert to pandas DataFrames
count_matrix_1 = gene_matrix_1.X.toarray()
count_matrix_2 = gene_matrix_2.X.toarray()

In [74]:
count_df_1 = pd.DataFrame(count_matrix_1, index=gene_matrix_1.obs_names, columns=gene_matrix_1.var_names)
count_df_2 = pd.DataFrame(count_matrix_2, index=gene_matrix_2.obs_names, columns=gene_matrix_2.var_names)

In [75]:
count_df_1.index = count_df_1.index + '_1'
count_df_2.index = count_df_2.index + '_2'

In [78]:
#Add metadata information
metadata_df_1 = pd.DataFrame(index=count_df_1.index)
metadata_df_1['Treatment'] = ['Treated w/AOA'] * len(gene_matrix_1.obs_names)

metadata_df_2 = pd.DataFrame(index=count_df_2.index)
metadata_df_2['Treatment'] = ['Untreated'] * len(gene_matrix_2.obs_names)

In [79]:
# Combine the two scATAC-seq count matrices
count_df = pd.concat([count_df_1, count_df_2], axis=0)
metadata_df = pd.concat([metadata_df_1, metadata_df_2], axis=0)

In [80]:
count_df

,DDX11L1,WASH7P,MIR6859-1,MIR1302-2HG,MIR1302-2,FAM138A,OR4G4P,OR4G11P,OR4F5,ENSG00000238009,...,MT-ND4,MT-TH,MT-TS2,MT-TL2,MT-ND5,MT-ND6,MT-TE,MT-CYB,MT-TT,MT-TP
AAAAAAAAAAAAAAAA_1,0,0,0,0,0,0,0,0,0,0,...,4,2,2,2,2,6,4,4,4,2
AAACAACGAACGAGCA_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
AAACAACGAAGAGGCT_1,0,0,0,0,0,0,0,0,0,0,...,9,8,9,9,14,8,8,11,8,2
AAACAACGAAGTCGGA_1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,4,4,1,0,4
AAACAACGACGCACTG_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,3,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGGATGAAGTCTC_2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
TTTGGGATGATGTGGG_2,0,0,0,0,0,0,0,0,0,0,...,9,8,8,8,12,8,7,9,9,1
TTTGGGATGCTCACGG_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,0
TTTGGGATGCTCTGGG_2,0,0,0,0,0,0,0,0,0,0,...,9,6,6,5,5,10,9,9,9,1


In [81]:
metadata_df

,Treatment
AAAAAAAAAAAAAAAA_1,Treated w/AOA
AAACAACGAACGAGCA_1,Treated w/AOA
AAACAACGAAGAGGCT_1,Treated w/AOA
AAACAACGAAGTCGGA_1,Treated w/AOA
AAACAACGACGCACTG_1,Treated w/AOA
...,...
TTTGGGATGAAGTCTC_2,Untreated
TTTGGGATGATGTGGG_2,Untreated
TTTGGGATGCTCACGG_2,Untreated
TTTGGGATGCTCTGGG_2,Untreated


In [59]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

In [82]:
dds = DeseqDataSet(
    counts=count_df,
    metadata=metadata_df,
    design_factors="Treatment",
    refit_cooks=True,
)

In [ ]:
# Perform differential accessibility analysis
dds.deseq2()
dds.get_results()

Fitting size factors...
/usr/local/lib/python3.10/site-packages/pydeseq2/dds.py:490: UserWarning: Every gene contains at least one zero, cannot compute log geometric means. Switching to iterative mode.
  self.fit_size_factors()


# Make observation names unique



In [ ]:
 # Check if any gene names are duplicated
has_duplicates = adata_concat.var_names.duplicated().any()

if has_duplicates:
    print("var_names are not unique.")
else:
    print("var_names are unique.")

In [ ]:
# Check if the number of unique gene names is equal to the total number of gene names
is_unique = len(adata_concat.var_names) == pd.Series(adata_concat.var_names).nunique()

if is_unique:
    print("var_names are unique.")
else:
    print("var_names are not unique.")

In [ ]:
# Check if any observations are duplicated
has_duplicates = adata_concat.obs_names.duplicated().any()

if has_duplicates:
    print("Observations are not unique.")
else:
    print("Observations are unique.")

In [ ]:
# Check if the number of unique observations is equal to the total number of observations
is_unique = len(adata_concat.obs_names) == pd.Series(adata_concat.obs_names).nunique()

if is_unique:
    print("Observations are unique.")
else:
    print("Observations are not unique.")

In [ ]:
adata_concat.var_names

In [ ]:
#Create unique observation names
experiment_names = ["Treated w/AOA", "Treated w/DON", "Treated w/DG", "Untreated"]

# Create a new column to store combined barcode and treatment information
adata_concat.obs['barcode_treatment'] = adata_concat.obs_names.astype(str)

# Modify the 'barcode_treatment' column
for i, treatment in enumerate(experiment_names):
    cell_indices = adata_concat.obs.index[adata_concat.obs["Treatment"] == treatment]
    adata_concat.obs.loc[cell_indices, 'barcode_treatment'] = adata_concat.obs.loc[cell_indices, 'barcode_treatment'] + "_" + str(i + 1)

In [ ]:
#Set 'barcode_treatment' as the new index:
adata_concat.obs_names = pd.Index(adata_concat.obs['barcode_treatment'])

In [ ]:
# Filter mitochondrial genes before highly variable gene selection
adata_concat.var['mt'] = adata_concat.var_names.str.startswith('MT-')
adata_filtered = adata_concat[:, ~adata_concat.var['mt']]

In [ ]:
# Calculate highly variable features
sc.pp.highly_variable_genes(adata_filtered, flavor='seurat', batch_key="Treatment")

In [ ]:
adata_filtered.var

Dispersion_norm represents the normalized dispersion of each gene.
Dispersion is a measure of how spread out the expression of a gene is across different cells.
Normalization adjusts the dispersion values to account for the relationship between mean expression and dispersion, ensuring that genes with similar mean expression levels are compared fairly.
Higher values of dispersions_norm indicate genes with higher variability or dispersion.
Dispersions_norm is a key factor used for ranking highly variable genes.
Genes with higher dispersions_norm values are considered more variable and are ranked higher.

In [ ]:
# Create a new 'highly_variable_rank' column
adata_filtered.var['highly_variable_rank'] = adata_filtered.var['dispersions_norm'].rank(ascending=False)

In [ ]:
# Sort genes by 'highly_variable_rank' in ascending order (lower rank = more variable)
adata_filtered.var = adata_filtered.var.sort_values(by='highly_variable_rank')

In [ ]:
# Print or access the ranked genes
adata_filtered.var

In [ ]:
# Subsample features (e.g., top 1000 highly variable genes)
adata_subset = adata_filtered[:, adata_filtered.var.index[:1000]] # Select the top 1000 genes

In [ ]:
adata_subset.var